# Fashion-MNIST 이미지 분류 모델

## 프로젝트 개요
- **목표**: Fashion-MNIST 데이터셋을 사용한 의류 이미지 10-class 분류
- **모델**: Convolutional Neural Network (CNN)
- **데이터**: 28x28 픽셀 흑백 의류 이미지
- **클래스**: 10가지 의류 카테고리

---

## 1. 라이브러리 및 의존성 Import

In [2]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow import keras
from keras import layers, models
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report
import pickle

print(f"TensorFlow Version: {tf.__version__}")

TensorFlow Version: 2.15.1


## 2. 전역 설정 및 상수 정의

In [3]:
# 모델 및 학습 히스토리 저장 경로 설정
model_path = "fashion_mnist_model.keras"  # 훈련된 모델 저장 파일
history_path = "fashion_mnist_history.bin"  # 학습 히스토리 저장 파일

# Fashion-MNIST 클래스 라벨 정의
class_names = ['T-shirt/top', 'Trouser', 'Pullover', 'Dress', 'Coat', 
               'Sandal', 'Shirt', 'Sneaker', 'Bag', 'Ankle boot']

print(f"모델 저장 경로: {model_path}")
print(f"히스토리 저장 경로: {history_path}")
print(f"분류 클래스: {len(class_names)}개")

모델 저장 경로: fashion_mnist_model.keras
히스토리 저장 경로: fashion_mnist_history.bin
분류 클래스: 10개


## 3. 데이터 로딩

Fashion-MNIST는 Keras에서 제공하는 표준 데이터셋입니다.
- **훈련 데이터**: 60,000개 이미지
- **테스트 데이터**: 10,000개 이미지  
- **이미지 크기**: 28×28 픽셀 (흑백)
- **클래스 수**: 10개 의류 카테고리

In [4]:
def load_data():
    """
    Fashion-MNIST 데이터셋을 로드하고 기본 정보를 출력합니다.
    
    Returns:
        tuple: (X_train, y_train, X_test, y_test)
    """
    # Keras에서 제공하는 Fashion-MNIST 데이터셋 로드
    (X_train, y_train), (X_test, y_test) = keras.datasets.fashion_mnist.load_data()
    
    # 데이터셋 형태 확인
    print("=== 데이터셋 정보 ===")
    print(f"훈련 이미지 shape: {X_train.shape}")
    print(f"훈련 레이블 shape: {y_train.shape}")
    print(f"테스트 이미지 shape: {X_test.shape}")
    print(f"테스트 레이블 shape: {y_test.shape}")
    
    # 레이블 분포 확인
    print(f"\n첫 10개 테스트 레이블: {y_test[:10]}")
    print(f"픽셀 값 범위: {X_train.min()} ~ {X_train.max()}")
    
    return X_train, y_train, X_test, y_test

## 4. 데이터 전처리

딥러닝 모델 학습을 위한 데이터 전처리 과정:
1. **정규화**: 픽셀 값을 0-1 범위로 스케일링 (0-255 → 0-1)
2. **차원 확장**: CNN을 위한 채널 차원 추가 (28, 28) → (28, 28, 1)
3. **데이터 타입 변환**: float32로 변환하여 메모리 효율성 향상

In [5]:
def preprocessing(X_train, y_train, X_test, y_test):
    """
    이미지 데이터를 딥러닝 모델에 적합하도록 전처리합니다.
    
    Args:
        X_train, y_train: 훈련 데이터
        X_test, y_test: 테스트 데이터
        
    Returns:
        tuple: 전처리된 (X_train, y_train, X_test, y_test)
    """
    print("=== 데이터 전처리 시작 ===")
    
    # 1. 정규화: 픽셀 값을 0-1 범위로 스케일링 
    X_train = X_train.astype("float32") / 255.0
    X_test = X_test.astype("float32") / 255.0
    print(f"정규화 완료: 픽셀 값 범위 {X_train.min():.1f} ~ {X_train.max():.1f}")
    
    # 2. CNN을 위한 채널 차원 추가 (28, 28) → (28, 28, 1)
    X_train = np.expand_dims(X_train, -1)
    X_test = np.expand_dims(X_test, -1)
    
    print(f"차원 확장 완료:")
    print(f"  - 훈련 데이터: {X_train.shape}")
    print(f"  - 테스트 데이터: {X_test.shape}")
    
    # 레이블은 sparse_categorical_crossentropy 사용으로 원핫인코딩 불필요
    print("레이블: sparse_categorical_crossentropy 사용으로 원핫인코딩 생략")
    
    return X_train, y_train, X_test, y_test

## 5. CNN 모델 아키텍처 구성

Fashion-MNIST 분류를 위한 Convolutional Neural Network 모델 설계:

### 모델 구조
1. **Convolutional Layers**: 특징 추출
   - Conv2D(32) + ReLU + MaxPooling2D
   - Conv2D(64) + ReLU + MaxPooling2D
2. **Fully Connected Layers**: 분류
   - Flatten → Dense(64) → Dense(32) → Dense(10)
3. **출력층**: Softmax 활성화 (10개 클래스 확률)

### 데이터 증강 (옵션)
- RandomFlip, RandomRotation, RandomZoom

In [6]:
def getModel(use_data_augmentation=False):
    """
    Fashion-MNIST 분류용 CNN 모델을 생성합니다.
    
    Args:
        use_data_augmentation (bool): 데이터 증강 사용 여부
        
    Returns:
        keras.Model: 컴파일된 CNN 모델
    """
    print("=== CNN 모델 구성 ===")
    
    # 데이터 증강 레이어 (선택적)
    data_augmentation = keras.Sequential([
        layers.RandomFlip("horizontal"),
        layers.RandomRotation(0.1),
        layers.RandomZoom(0.2)
    ], name="data_augmentation")
    
    # CNN 모델 아키텍처 구성
    model_layers = []
    
    # 데이터 증강 추가 (옵션)
    if use_data_augmentation:
        model_layers.append(data_augmentation)
        print("데이터 증강 레이어 추가됨")
    
    # Convolutional 블록 1
    model_layers.extend([
        layers.Conv2D(32, kernel_size=(3,3), activation="relu", 
                     input_shape=(28,28,1), name="conv2d_1"),
        layers.MaxPooling2D(pool_size=(2,2), name="maxpool_1"),
    ])
    
    # Convolutional 블록 2  
    model_layers.extend([
        layers.Conv2D(64, kernel_size=(3,3), activation="relu", name="conv2d_2"),
        layers.MaxPooling2D(pool_size=(2,2), name="maxpool_2"),
    ])
    
    # Fully Connected 블록
    model_layers.extend([
        layers.Flatten(name="flatten"),
        layers.Dense(64, activation='relu', name="dense_1"),
        layers.Dense(32, activation='relu', name="dense_2"), 
        layers.Dense(10, activation='softmax', name="output")
    ])
    
    # 모델 생성
    model = keras.Sequential(model_layers, name="fashion_mnist_cnn")
    
    # 모델 컴파일
    model.compile(
        optimizer='adam',
        loss='sparse_categorical_crossentropy',
        metrics=['accuracy']
    )
    
    print("모델 컴파일 완료")
    print(f"데이터 증강: {'사용' if use_data_augmentation else '미사용'}")
    model.summary()
    
    return model

## 6. 모델 학습 및 훈련

### 학습 설정
- **Optimizer**: Adam 
- **Loss Function**: Sparse Categorical Crossentropy
- **Metrics**: Accuracy
- **Validation Split**: 20% (훈련 데이터의 20%를 검증용으로 사용)
- **Early Stopping**: 검증 손실이 5 에포크 동안 개선되지 않으면 조기 종료

In [7]:
def train_model(model, X_train, y_train, X_test, y_test, epochs=100):
    """
    모델을 훈련하고 결과를 저장합니다.
    
    Args:
        model: 훈련할 Keras 모델
        X_train, y_train: 훈련 데이터
        X_test, y_test: 테스트 데이터  
        epochs (int): 최대 훈련 에포크 수
        
    Returns:
        keras.callbacks.History: 학습 히스토리
    """
    print("=== 모델 학습 시작 ===")
    
    # 콜백 설정: 조기 종료
    early_stopping = keras.callbacks.EarlyStopping(
        monitor='val_loss',
        patience=5,
        restore_best_weights=True,
        verbose=1
    )
    
    print(f"학습 설정:")
    print(f"  - 최대 에포크: {epochs}")
    print(f"  - 검증 분할: 20%")
    print(f"  - 조기 종료: val_loss 5 에포크 patience")
    
    # 모델 훈련
    history = model.fit(
        X_train, y_train,
        epochs=epochs,
        validation_split=0.2,
        callbacks=[early_stopping],
        verbose=1
    )
    
    # 모델 및 학습 히스토리 저장
    print(f"\n=== 모델 저장 ===")
    model.save(model_path)
    print(f"모델 저장 완료: {model_path}")
    
    with open(history_path, 'wb') as f:
        pickle.dump(history.history, f)
    print(f"히스토리 저장 완료: {history_path}")
    
    return history

## 7. 전체 파이프라인 실행

데이터 로딩부터 모델 훈련까지의 전체 과정을 순차적으로 실행합니다.

In [9]:
# 전체 머신러닝 파이프라인 실행
print("=" * 60)
print("Fashion-MNIST CNN 모델 훈련 파이프라인 시작")
print("=" * 60)

# 1. 데이터 로딩
X_train, y_train, X_test, y_test = load_data()

# 2. 데이터 전처리  
X_train, y_train, X_test, y_test = preprocessing(X_train, y_train, X_test, y_test)

# 3. 모델 생성
model = getModel(use_data_augmentation=False)  # 데이터 증강 비활성화

# 4. 모델 훈련
history = train_model(model, X_train, y_train, X_test, y_test, epochs=100)

print("=" * 60)
print("모델 훈련 완료!")
print("=" * 60)

Fashion-MNIST CNN 모델 훈련 파이프라인 시작
=== 데이터셋 정보 ===
훈련 이미지 shape: (60000, 28, 28)
훈련 레이블 shape: (60000,)
테스트 이미지 shape: (10000, 28, 28)
테스트 레이블 shape: (10000,)

첫 10개 테스트 레이블: [9 2 1 1 6 1 4 6 5 7]
픽셀 값 범위: 0 ~ 255
=== 데이터 전처리 시작 ===
정규화 완료: 픽셀 값 범위 0.0 ~ 1.0
차원 확장 완료:
  - 훈련 데이터: (60000, 28, 28, 1)
  - 테스트 데이터: (10000, 28, 28, 1)
레이블: sparse_categorical_crossentropy 사용으로 원핫인코딩 생략
=== CNN 모델 구성 ===
모델 컴파일 완료
데이터 증강: 미사용
Model: "fashion_mnist_cnn"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_1 (Conv2D)           (None, 26, 26, 32)        320       
                                                                 
 maxpool_1 (MaxPooling2D)    (None, 13, 13, 32)        0         
                                                                 
 conv2d_2 (Conv2D)           (None, 11, 11, 64)        18496     
                                                                 
 maxpool_2 (MaxPooling2D

## 8. 모델 평가 및 성능 분석

훈련된 모델의 성능을 다양한 지표로 평가합니다:

### 평가 지표
1. **기본 지표**: 테스트셋 손실 및 정확도
2. **혼동 행렬**: 클래스별 분류 성능 시각화
3. **분류 보고서**: 정밀도, 재현율, F1-점수 등 상세 지표
4. **클래스별 성능**: 각 의류 카테고리별 분류 성능

In [10]:
def evaluate_model(X_test, y_test):
    """
    저장된 모델을 로드하여 테스트 데이터로 성능을 평가합니다.
    
    Args:
        X_test: 테스트 이미지 데이터
        y_test: 테스트 레이블 데이터
    """
    print("=" * 60)
    print("모델 평가 시작")
    print("=" * 60)
    
    # 저장된 모델과 히스토리 로드
    model = keras.models.load_model(model_path)
    print(f"모델 로드 완료: {model_path}")
    
    with open(history_path, 'rb') as f:
        history = pickle.load(f)
    print(f"히스토리 로드 완료: {history_path}")
    
    # === 1. 예측 수행 ===
    print("\n=== 예측 수행 ===")
    y_pred_proba = model.predict(X_test, verbose=0)  # 예측 확률
    print(f"예측 확률 shape: {y_pred_proba.shape}")
    print(f"첫 번째 샘플 예측 확률: {y_pred_proba[0]}")
    
    # 확률에서 클래스 인덱스로 변환
    y_pred = np.argmax(y_pred_proba, axis=1)
    print(f"\n예측 클래스 (처음 10개): {y_pred[:10]}")
    print(f"실제 클래스 (처음 10개): {y_test[:10]}")
    
    # === 2. 기본 성능 지표 ===
    print("\n=== 기본 성능 지표 ===")
    loss, accuracy = model.evaluate(X_test, y_test, verbose=0)
    print(f"테스트 손실 (Loss): {loss:.4f}")
    print(f"테스트 정확도 (Accuracy): {accuracy:.4f} ({accuracy*100:.2f}%)")
    
    # === 3. 혼동 행렬 ===
    print("\n=== 혼동 행렬 ===")
    cm = confusion_matrix(y_test, y_pred)
    print(cm)
    
    # === 4. 상세 분류 보고서 ===
    print("\n=== 분류 보고서 ===")
    report = classification_report(
        y_test, y_pred, 
        target_names=class_names,
        digits=4
    )
    print(report)
    
    # === 5. 요약 통계 ===
    print("\n=== 성능 요약 ===")
    correct_predictions = np.sum(y_pred == y_test)
    total_predictions = len(y_test)
    print(f"전체 테스트 샘플: {total_predictions:,}")
    print(f"올바른 예측: {correct_predictions:,}")
    print(f"잘못된 예측: {total_predictions - correct_predictions:,}")
    print(f"최종 정확도: {correct_predictions/total_predictions:.4f}")
    
    return model, history, y_pred, y_pred_proba

## 9. 모델 평가 실행

저장된 모델을 로드하여 테스트 데이터셋으로 최종 성능을 평가합니다.

In [11]:
# 모델 평가 실행
model_eval, history_eval, predictions, prediction_probabilities = evaluate_model(X_test, y_test)

모델 평가 시작
모델 로드 완료: fashion_mnist_model.keras
히스토리 로드 완료: fashion_mnist_history.bin

=== 예측 수행 ===
예측 확률 shape: (10000, 10)
첫 번째 샘플 예측 확률: [1.9697832e-06 2.9713748e-08 6.0883741e-07 5.1347211e-08 4.6203857e-07
 1.0105026e-03 2.0627771e-05 1.1649434e-03 1.1705238e-07 9.9780077e-01]

예측 클래스 (처음 10개): [9 2 1 1 6 1 4 6 5 7]
실제 클래스 (처음 10개): [9 2 1 1 6 1 4 6 5 7]

=== 기본 성능 지표 ===
테스트 손실 (Loss): 0.2636
테스트 정확도 (Accuracy): 0.9067 (90.67%)

=== 혼동 행렬 ===
[[879   0  21  12   1   0  82   0   5   0]
 [  2 970   0  20   2   0   5   0   1   0]
 [ 15   1 872  10  38   0  62   0   2   0]
 [ 22   2   8 924  14   0  30   0   0   0]
 [  0   0  67  30 814   0  87   0   2   0]
 [  1   0   0   0   0 981   0  14   1   3]
 [132   0  77  24  40   0 716   0  11   0]
 [  0   0   0   0   0   6   0 976   3  15]
 [  2   0   3   5   0   2   5   1 982   0]
 [  1   0   0   0   0   7   0  37   2 953]]

=== 분류 보고서 ===
              precision    recall  f1-score   support

 T-shirt/top     0.8340    0.8790    0.8559    